In [12]:
import pandas as pd
import psycopg2
import json


In [6]:
def connect():
    conn = None
    try:
        print('Connecting...')
        conn = psycopg2.connect(
            host='10.10.248.108',
            database='postgres',
            user='eatin',
            password='eatin')
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    print('Connection Successful')

    return conn


def execute_select(conn, query, column_names):
    cursor = conn.cursor()
    data = pd.DataFrame()
    try:
        cursor.execute(query)
        data = cursor.fetchall()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    cursor.close()

    return pd.DataFrame(data, columns=column_names)



In [7]:
RECIPE_COLUMNS = ['index',
'recipe_title',
'url',
'record_health',
'vote_count',
'rating',
'description',
'cuisine',
'course',
'diet',
'prep_time',
'cook_time',
'ingredients',
'instructions',
'author',
'tags',
'category',
'image',
'difficulty',
'total_time']

In [8]:
query = "select * from recipes order by vote_count desc, rating desc limit 100;"
conn = connect()
df = execute_select(conn, query, RECIPE_COLUMNS)
recipes_json = json.loads(df.to_json(orient='records'))
recipes = [{'name': 'for_you', 'recipes': recipes_json[:13]},
           {'name': 'other', 'recipes': recipes_json[14:99]}]
conn.close()



Connecting...
Connection Successful


In [46]:
recipes[1]

{'name': 'other',
 'recipes': [{'index': 1674,
   'recipe_title': 'Green Chutney Recipe - Coriander Mint Chutney Recipe',
   'url': 'https://www.archanaskitchen.com/green-chutney-recipe-coriander-mint-chutney-recipe',
   'record_health': 'good',
   'vote_count': 29632,
   'rating': 4.5859721053,
   'description': ' also known as the  . This chutney is made from fresh coriander and mint leaves along with green chillies, ginger and lemon and a dash of sugar that gives this chutney a unique flavor. Each household and region of India have their own way of making the green chutney. This version has no onion and no garlic and tastes fresh and delicious.  This Dhaniya Pudina Chutney recipe is a favorite Indian accompaniment for Samosas, Dhoklas, Vada Pavs, Bhel that are incomplete without this chutney. The Dhaniya Pudina Chutney is also used in a variety of chaat dishes and as sandwich spread. Dhaniya Pudina Chutney can be stored in the fridge in an airtight container up to a week and so you 

In [22]:
top_categories_query = "SELECT \
  category, \
  COUNT(*) AS recipe_count, \
  SUM(vote_count) AS total_votes, \
  AVG(rating) AS average_rating, \
  (0.4 * SUM(vote_count) + 0.4 * AVG(rating) + 0.2 * COUNT(*)) AS popularity_score \
FROM recipes \
GROUP BY category \
ORDER BY popularity_score DESC \
LIMIT 10;"

top_recipes_for_category_query = "select * from recipes \
where category = '{}' \
order by (vote_count * rating) desc \
limit 13;"


In [17]:
conn = connect()
df = execute_select(conn, top_categories_query, ['category', 'recipe_count', 'total_votes', 'average_rating', 'popularity_score'])
categories_json = json.loads(df.to_json(orient='records'))
# recipes = [{'name': 'for_you', 'recipes': recipes_json[:13]},
#            {'name': 'other', 'recipes': recipes_json[14:99]}]
conn.close()


Connecting...
Connection Successful


In [29]:
categories_json

[{'category': 'Sabzi Recipes',
  'recipe_count': 1037,
  'total_votes': 2580361.0,
  'average_rating': 4.4208918849,
  'popularity_score': 1032353.568356754},
 {'category': 'Indian Snack Recipes',
  'recipe_count': 707,
  'total_votes': 1630854.0,
  'average_rating': 4.3718326617,
  'popularity_score': 652484.7487330646},
 {'category': 'Sweet Recipes (Indian Mithai / Indian Dessert)',
  'recipe_count': 432,
  'total_votes': 957021.0,
  'average_rating': 4.3473248875,
  'popularity_score': 382896.538929955},
 {'category': 'Chicken Recipes',
  'recipe_count': 134,
  'total_votes': 601673.0,
  'average_rating': 4.5065146809,
  'popularity_score': 240697.8026058724},
 {'category': 'Dal Recipes',
  'recipe_count': 217,
  'total_votes': 527594.0,
  'average_rating': 4.4842997831,
  'popularity_score': 211082.7937199132},
 {'category': 'Poriyal Recipes (South Indian Sabzi)',
  'recipe_count': 191,
  'total_votes': 500991.0,
  'average_rating': 4.5643943634,
  'popularity_score': 200436.425757

In [31]:
kaki = [baki(category) for category in df['category']]

Connecting...
Connection Successful
Connecting...
Connection Successful
Connecting...
Connection Successful
Connecting...
Connection Successful
Connecting...
Connection Successful
Connecting...
Connection Successful
Connecting...
Connection Successful
Connecting...
Connection Successful
Connecting...
Connection Successful
Connecting...
Connection Successful


In [30]:
def baki(category):
    query = top_recipes_for_category_query.format(category)
    conn = connect()
    df = execute_select(conn, query, RECIPE_COLUMNS)
    recipes_json = json.loads(df.to_json(orient='records'))
    # recipes = [{'name': 'for_you', 'recipes': recipes_json[:13]},
    #            {'name': 'other', 'recipes': recipes_json[14:99]}]
    section = {'name': category, 'recipes': recipes_json}
    conn.close()
    return section

In [47]:
recipes.append(kaki)

In [53]:
recipes[2]

[{'name': 'Sabzi Recipes',
  'recipes': [{'index': 2131,
    'recipe_title': 'Punjabi Style Tinda Sabzi Recipe',
    'url': 'https://www.archanaskitchen.com/punjabi-style-tinda-sabji-recipe',
    'record_health': 'good',
    'vote_count': 42785,
    'rating': 4.4209733277,
    'description': ' is a flavourful sabzi, perfect for a quick weekday lunch. The humble tinda when flavoured with garam masala and other spices is sure to win over the most ardent Tinda skeptics. Quick and easy Tinda sabzi along with Chapati makes for a delicious lunch box meal. Serve   along with  ,  and   for a weekday lunch. If you like this recipe, you can also try other   such as',
    'cuisine': 'Punjabi',
    'course': 'Lunch',
    'diet': 'Vegetarian',
    'prep_time': 5,
    'cook_time': 15,
    'ingredients': '{"8 Tinda (Apple Gourd), peeled, sliced and hard seeds removed","1 Onion, finely chopped","4 cloves Garlic, minced","1 Tomato, chopped","2 tablespoons Tomato, paste","1/2 teaspoon Coriander Powder (

In [54]:
dfkaki = pd.read_parquet('../dataset/recipes-formatted.parquet.gzip')

In [63]:
dfkaki

,recipe_title,url,record_health,vote_count,rating,description,cuisine,course,diet,prep_time,cook_time,ingredients,instructions,author,tags,category,image,difficulty,total_time
0,Roasted Peppers And Mushroom Tortilla Pizza Re...,https://www.archanaskitchen.com/roasted-pepper...,good,434,4.958525,is a quicker version pizza to satisfy your cr...,Mexican,Dinner,Vegetarian,15,15,"[2 Tortillas, (wheat/regular), 1/4 cup Homemad...",[To begin making the Roasted Peppers And Mushr...,Divya Shivaraman,"[Party Food Recipes, Tea Party Recipes, Mushro...",Pizza Recipes,https://www.archanaskitchen.com/images/archana...,Easy,30
1,Thakkali Gotsu Recipe | Thakkali Curry | Spicy...,https://www.archanaskitchen.com/tomato-gotsu-r...,good,3423,4.932223,also known as the is a quick and easy to ma...,South Indian Recipes,Lunch,Vegetarian,10,20,"[1 tablespoon Sesame (Gingelly) Oil, 1/2 teasp...",[To begin making Tomato Gotsu Recipe/ Thakkali...,Archana Doshi,"[Vegetarian Recipes, Tomato Recipes, South Ind...",Indian Curry Recipes,https://www.archanaskitchen.com/images/archana...,Easy,30
2,Spicy Grilled Pineapple Salsa Recipe,https://www.archanaskitchen.com/spicy-grilled-...,good,2091,4.945959,Spicy Grilled Pineapple Salsa is a simple reci...,Mexican,Side Dish,Vegetarian,10,0,"[1 tablespoon Extra Virgin Olive Oil, 1 Pineap...",[To begin making the Spicy Grilled Pineapple S...,Archana's Kitchen,"[Party Starter & Appetizer Recipes, Pineapple ...",Mexican Recipes,https://www.archanaskitchen.com/images/archana...,Easy,10
3,Karwar Style Dali Thoy Recipe - Toor dal Curry,https://www.archanaskitchen.com/dali-thoy-reci...,good,990,4.888889,The is a quintessential of Konkani dish whic...,Coastal Karnataka,Side Dish,High Protein Vegetarian,5,20,"[1 cup Arhar dal (Split Toor Dal), 1/2 teaspoo...",[To prepare Karwar Style Dali Thoy Recipe (Too...,Jyothi Rajesh,"[Side Dish Recipes, South Indian Recipes, Indi...",Indian Curry Recipes,https://www.archanaskitchen.com/images/archana...,Easy,25
4,Rajma Kofta In Milk And Poppy Seed Gravy Recipe,https://www.archanaskitchen.com/rajma-kofta-in...,good,345,4.828986,Koftas are traditional Indian recipes mostly w...,North Indian Recipes,Side Dish,High Protein Vegetarian,20,30,"[1/2 cup Rajma (Large Kidney Beans), 1/4 cup C...",[To begin making Rajma Kofta In Milk And Poppy...,RUBY PATHAK,"[Side Dish Recipes, Indian Lunch Recipes, Offi...",Kofta Recipes,https://www.archanaskitchen.com/images/archana...,Easy,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8004,Zucchini and Carrot Pasta Recipe,https://www.archanaskitchen.com/zucchini-and-c...,good,738,4.934959,is a healthy recipe which can be served over ...,Italian Recipes,Dinner,Vegetarian,10,30,"[2 cups Spirali pasta, boiled, (add some salt ...",[To begin with Zucchini And Carrot Pasta Recip...,Aarti Sharma,"[Vegetarian Recipes, Carrot Recipes, Zucchini ...",Pasta Recipes,https://www.archanaskitchen.com/images/archana...,Easy,40
8005,Hyderabadi Khatti Dal Recipe (Hyderabadi Tangy...,https://www.archanaskitchen.com/hyderabadi-kha...,good,5173,4.903151,Hyderabadi Khatti Dal is a delicious tangy dal...,Hyderabadi,Lunch,High Protein Vegetarian,20,40,"[1 cup Arhar dal (Split Toor Dal), 1 Tomato, f...",[To begin making the Hyderabadi Khatti Dal Rec...,Farrukh Aziz Ansari,"[Gluten Free Recipes, Indian Diabetic Recipes]",Indian Curry Recipes,https://www.archanaskitchen.com/images/archana...,Easy,60
8006,Karwar Style Muga Ambat Recipe (Moong Sprouts ...,https://www.archanaskitchen.com/karwar-style-m...,good,1202,4.946755,is a curry recipe made from 'Muga' - which re...,Karnataka,Side Dish,High Protein Vegetarian,10,15,"[1 cup Green Moong Sprouts, 1 tablespoon Tamar...",[To prepare Karwar Style Muga Ambat Recipe (Mo...,Archana's Kitchen,"[High Protein Vegetarian Diet Recipes, South I...",Indian Curry Recipes,https://www.archanaskitchen.com/images/archana...,Easy,25
8007,"Beetroot, Amla And Pudina Juice Recipe | Mint ...",https://www.archanaskitchen.com

In [62]:
dfkaki[dfkaki['course'] == 'Dinner']

,recipe_title,url,record_health,vote_count,rating,description,cuisine,course,diet,prep_time,cook_time,ingredients,instructions,author,tags,category,image,difficulty,total_time
0,Roasted Peppers And Mushroom Tortilla Pizza Re...,https://www.archanaskitchen.com/roasted-pepper...,good,434,4.958525,is a quicker version pizza to satisfy your cr...,Mexican,Dinner,Vegetarian,15,15,"[2 Tortillas, (wheat/regular), 1/4 cup Homemad...",[To begin making the Roasted Peppers And Mushr...,Divya Shivaraman,"[Party Food Recipes, Tea Party Recipes, Mushro...",Pizza Recipes,https://www.archanaskitchen.com/images/archana...,Easy,30
11,Cheesy Rice Poppers Recipe,https://www.archanaskitchen.com/cheesy-rice-po...,good,2431,4.951049,Cheesy Rice Poppers Recipe is a lovely finger ...,Continental,Dinner,Vegetarian,10,10,"[2 tablespoons Butter (Salted), 4 cloves Garli...",[To begin making the Cheesy Rice Poppers Recip...,Archana's Kitchen,"[Party Food Recipes, Tea Party Recipes, Cheese...",Continental Party Appetizer & Starter Recipes,https://www.archanaskitchen.com/images/archana...,Easy,20
13,Goan Sorak Curry Recipe,https://www.archanaskitchen.com/goan-sorak-cur...,good,1967,4.853076,"is a sweet, spicy, tangy and delicious curry ...",Goan Recipes,Dinner,Vegetarian,15,15,"[1 tablespoon Oil, 3 Kokum (Malabar Tamarind),...","[To begin making the Goan Sorak Curry Recipe, ...",Archana's Kitchen,"[Vegetarian Recipes, Indian Lunch Recipes, Goa...",Indian Curry Recipes,https://www.archanaskitchen.com/images/archana...,Easy,30
20,Creamy Chicken Pesto Spaghetti Pasta Recipe,https://www.archanaskitchen.com/creamy-chicken...,good,1162,4.904475,The is a lip smacking pasta recipe that is m...,Italian Recipes,Dinner,High Protein Non Vegetarian,10,20,"[2 tablespoons Extra Virgin Olive Oil, 4 Chick...",[To begin making the Creamy Chicken Pesto Past...,Anshu Pande,"[Party Food Recipes, Healthy Recipes, Italian ...",Pasta Recipes,https://www.archanaskitchen.com/images/archana...,Easy,30
39,Spicy Pressure Cooker Mutton Curry Recipe,https://www.archanaskitchen.com/spicy-and-quic...,good,932,4.903433,Spicy Pressure Cooker Mutton Curry Recipe is ...,Indian,Dinner,Non Vegeterian,30,30,"[750 grams Mutton, on the bone, Salt, 1 teaspo...",[To begin making Spicy Pressure Cooker Mutton ...,Archana's Kitchen,"[Party Food Recipes, Indian Lunch Recipes, Wee...",Mutton Recipes,https://www.archanaskitchen.com/images/archana...,Easy,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7982,Grilled Vegetable Empanadas Recipe,https://www.archanaskitchen.com/grilled-vegeta...,good,693,4.914863,Grilled Vegetable Empanada is a grilled vegeta...,European,Dinner,Vegetarian,15,60,"[1/2 cup All Purpose Flour (Maida), 1/4 cup Wh...","[To start making Grilled Vegetable Empanadas, ...",Archana's Kitchen,"[Party Food Recipes, Party Starter & Appetizer...",Continental Party Appetizer & Starter Recipes,https://www.archanaskitchen.com/images/archana...,Easy,75
7988,Ukad Recipe (Maharashtrian Rice Flour Porridge),https://www.archanaskitchen.com/ukad-recipe-ma...,good,2324,4.905766,Ukad really has no English or a Hindi word as ...,Maharashtrian Recipes,Dinner,Vegetarian,5,5,"[1 tablespoon Oil, 1/2 teaspoon Mustard seeds ...","[To begin making the Ukad Recipe, prep all the...",Devaki Sonawala,"[Dahi Recipes (Curd/Yogurt), Rice Flour Recipes]",Variety Rice (Mixed Rice/ Flavoured Rice Recipes),https://www.archanaskitchen.com/images/archana...,Easy,10
8000,Buttered Herbed Rice Recipe,https://www.archanaskitchen.com/buttered-herbe...,good,15018,4.919497,is one of the easiest rice dish to make. It i...,Continental,Dinner,Vegetarian,20,20,"[2 cups Basmati rice, 1 teaspoon Garlic, finel...","[To begin making Buttered Herbed rice, heat b...",Farrukh Aziz Ansari,"[Indian Lunch Recipes, Office Lunch Box Recipe...",Continental Food Recipes,https://www.archanaskitchen.com/images/archana...,Easy,40
8002,Vankaya Pakodi Koora Recipe - Andhra Style Bri...,https://www.archanaskitchen.com/vank